In [4]:
import cv2 as cv
import numpy as np
import math
from matplotlib import pyplot as plt 

IMAGE_SIZE = 512

#read image
img = cv.imread('chessboardclean.jpg')
hh, ww = img.shape[:2]
#pick scaling dactor for IMAGE_SIZE
scaling_factor = hh if hh > ww else ww
scaling_factor =  IMAGE_SIZE/scaling_factor
#reescaling image
img = cv.resize(img, None, fx= scaling_factor, fy= scaling_factor, interpolation= cv.INTER_LINEAR)


#threshold on green
#defining lower and upper limits
lower = np.array([130, 130, 130])
upper = np.array([255, 255, 255])

# Create mask to only select black
thresh = cv.inRange(img, lower, upper)
cv.imwrite("thresh.jpg", thresh)

#generating boundary by threshold
try : contours,hierarchy = cv.findContours(image=thresh, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_NONE)
except : print('error')
for cnt in contours:
    # draw contours on the original image
    image_copy = img.copy()
    cv.drawContours(image=image_copy, contours=cnt, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)
    
    x,y,w,h = cv.boundingRect(cnt)
    area = cv.contourArea(cnt)
    #see if the width and height match the chessboard rectangle
    if w>290 and h>290 and area<95000 and area>85000:
        contour = cnt
        print('area=' + str(cv.contourArea(cnt)))
        break

#crop image by contour
image_copy = img.copy()
cropped = image_copy[y:y+h, x:x+w]
cv.imshow('cropped image', cropped)
cv.imwrite('contourscropped.jpg', cropped)
cv.waitKey(0)
cv.destroyAllWindows()

# draw contours on the original image
image_copy = img.copy()
cv.drawContours(image=image_copy, contours=contour, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)
                
# see the results
cv.imshow('None approximation', image_copy)
cv.waitKey(0)
cv.imwrite('contours_none_image1.jpg', image_copy)
cv.destroyAllWindows()

area=91659.0


In [5]:
#invert morp image
mask = 255 - thresh

# apply morphology to remove noises and letters
kernel = cv.getStructuringElement(cv.MORPH_RECT, (5,5))
morph = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)



# apply mask to image
result = cv.bitwise_and(img, img, mask=morph)

cv.imshow('thresh', thresh)
cv.imshow('morph', morph)
cv.imshow('mask', mask)
cv.imshow('result', result)
cv.waitKey(0)
cv.destroyAllWindows()

In [64]:

src = cv.cvtColor(result, cv.COLOR_BGR2GRAY)
dst = cv.Canny(src, 50, 200, None, 3)
cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)

lines = cv.HoughLines(dst, 1, np.pi / 180, 120, None, 0, 0)

if lines is not None:
    print(len(lines))
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv.line(cdst, pt1, pt2, (0, 0, 255), 3, cv.LINE_AA)
        #cv.imshow("line:", cdst)
        #cv.waitKey()
        #cv.destroyAllWindows()
cv.imshow("Detected Lines (in red) - Probabilistic Line Transform", cdst)
cv.waitKey()
cv.destroyAllWindows()


15
